In [3]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

tf.enable_v2_behavior()

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [4]:

model.fit(
    ds_train,
    epochs=2,
    validation_data=ds_test,
)

# saving model
model.save('model.savedmodel')


Epoch 1/2
469/469 [==============================] - 8s 6ms/step - loss: 0.6150 - sparse_categorical_accuracy: 0.8270 - val_loss: 0.2042 - val_sparse_categorical_accuracy: 0.9409
Epoch 2/2
469/469 [==============================] - 2s 3ms/step - loss: 0.1869 - sparse_categorical_accuracy: 0.9468 - val_loss: 0.1445 - val_sparse_categorical_accuracy: 0.9571
INFO:tensorflow:Assets written to: model.savedmodel/assets


INFO:tensorflow:Assets written to: model.savedmodel/assets


In [5]:
model.input

<KerasTensor: shape=(None, 28, 28, 1) dtype=float32 (created by layer 'flatten_2_input')>

In [6]:
model.output

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_5')>

# 

In [14]:
from PIL import Image

# pre-processing
def preprocessing():
    INPUT_SHAPE = (28, 28)
    '''
    Return (1, 28, 28, 1) with FP32 input from image
    '''
    img = Image.open('/content/7.png').convert('L')
    img = img.resize(INPUT_SHAPE)
    imgArr = np.asarray(img) / 255
    imgArr = np.expand_dims(imgArr[:, :, np.newaxis], 0)
    imgArr = imgArr.astype(np.float32)
    print(imgArr.shape)
    # print(imgArr)
    return imgArr

In [17]:
import numpy as np

# preprocess
x = preprocessing()

# x = np.random.rand(1, 28, 28, 1)
prediction = model.predict(x)
print(prediction)


(1, 28, 28, 1)
[[ -0.31259698  -5.9868555    2.9017212    3.748668    -6.3194785
   -1.1580417  -10.198615     9.290149    -0.31000635   0.17650017]]


# Load Model and how to get model params for config.pbtxt

In [10]:
from keras.models import load_model

loaded_model = load_model('/content/model.savedmodel')
loaded_model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_5')>

In [11]:
loaded_model.input

<KerasTensor: shape=(None, 28, 28, 1) dtype=float32 (created by layer 'flatten_2_input')>

In [12]:
loaded_model.output

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_5')>